In [48]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [49]:
maxProductPrice = 400000

Information about customers

In [50]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_partworth_dict = saedfsc.customers.set_index('Name')['PartworthUtilityWeights'].to_dict()
name_weights_perf_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWtsPartworth = {c : np.fromstring(name_weights_partworth_dict[c].strip('[]'), sep=',') for c in name_weights_partworth_dict}
cWtsPerf = {c : np.fromstring(name_weights_perf_dict[c].strip('[]'), sep=',') for c in name_weights_perf_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [51]:
def getTotalUtilityForCustomer(car : COTSCar, c : str, utilFn = 'partworth'):
    total_utility = 0
    pricePerf = (maxProductPrice - car.price) / maxProductPrice # Normalized price (0 is best, 1 is worst)
    if utilFn == 'partworth':
        product_utility = car.partworth_objectives(weights=cWtsPartworth[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility - cPriceFocus[c]*pricePerf
    elif utilFn == 'performance':
        product_utility = car.objectives(weights=cWtsPerf[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility + cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar], utilFn = 'partworth'): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c, utilFn)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c, utilFn) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [52]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[5, 7, 0, 7, 11, 1, 5, 1, 7, 2, 2, 37, 0, 2, 1...","$4,914.85","$4,468.05"
1,Competitor Car 1,"[4, 10, 9, 8, 12, 4, 3, 12, 30, 2, 48, 153, 0,...","$110,221.47","$100,201.34"
2,Competitor Car 2,"[3, 11, 4, 5, 5, 3, 3, 16, 26, 1, 13, 51, 1, 1...","$24,974.52","$22,704.11"


## Partworth Utility

Create a random car and compute market share

In [53]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 9,323.97
Cost: $ 8,476.34
Total expected profit: $ 44,691,069.92
Market information for our car:


,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-1.966241,0.476676,223.561197
1,CustomerType2,929,-1.014508,0.275714,256.138526
2,CustomerType3,294,-1.765798,0.381552,112.176288
3,CustomerType4,834,-1.599019,0.422365,352.252792
4,CustomerType5,733,-1.947100,0.427901,313.651345
5,CustomerType6,35,-1.586768,0.321175,11.241117
6,CustomerType7,239,-1.798625,0.289480,69.185812
7,CustomerType8,564,-1.765243,0.376613,212.409720
8,CustomerType9,927,-1.927681,0.395932,367.028562
9,CustomerType10,366,-2.158297,0.495084,181.200671


## Performance Utility

In [57]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
print("COTSCar vec:", product.get_vec())
print("Car vec:", product.car.get_vec())
print("Cost: $", format(product.car.cost(), ",.2f"))
product.set_vec(carDesignVec)
print("vec:", product.car.get_vec())
print("vec 2:", product.get_vec())
print("Cost: $", format(product.cost(), ",.2f"))
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c, 'performance') for c in customers]
market_shares = [getMarketShare(product, c, competitors, 'performance') for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors, 'performance')*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

COTSCar vec: [12, 9, 11, 5, 8, 5, 4, 9, 27, 3, 13, 90, 1, 0, 42, 84, 94]
Car vec: [0.3625, 0.15, 0.3926990815, 0.1125, 0.275, 1.65, 0.3926990815, 0.1125, 0.30000000000000004, 0.225, 0.3926990815, 0.894, 0.894, 0.7796592890146228, 2.347943283045604, 0.7693047262366127, 0.0075194955679045265, 0.3731208963514925, 0.4275070278407457, 10, 0, 10, 12, 6, 2, 5, 11, 18, 0, 2.202544777220481, 0.7613223493539524, 0.13020430160340302, 0.13478537792399203, 0.2720971788396407, 0.4702267442809723, 0.2920152781289319, 0.2857718325103866, 0.40252506860703263, 0.3174336941808231]
Cost: $ 21,762.22
vec: [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4, 0.3731208963514925, 0.4275070278407457, 10, 0, 10, 12, 6, 2, 5, 11, 18, 0, 2.202544777220481, 0.7613223493539524, 0.13020430160340302, 0.13478537792399203, 0.2720971788396407, 0.4702267442809723, 0.2920152781289319, 0.2857718325103866, 0.40252506860703263, 0.3174336941808231]
vec 2: [12, 9, 11, 5, 8, 5, 4, 9, 27, 3, 13, 90, 1, 0, 42, 84, 94]
Cost:

,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-1587.978976,1.003718,470.743856
1,CustomerType2,929,-149.995335,1.020154,947.723244
2,CustomerType3,294,-175.362170,1.028098,302.260826
3,CustomerType4,834,-341.014992,1.014477,846.073812
4,CustomerType5,733,-496.681525,1.008907,739.528856
5,CustomerType6,35,-910.926150,1.007551,35.264287
6,CustomerType7,239,-1077.770300,1.003880,239.927392
7,CustomerType8,564,-576.505613,1.006056,567.415476
8,CustomerType9,927,-737.850561,1.006813,933.315992
9,CustomerType10,366,-787.525507,1.007256,368.655524
